In [1]:
from ipyparallel import Client
rc = Client(profile='pucv-cluster')
len(rc)

95

In [2]:
bview = rc.load_balanced_view()
dview = rc[:]

In [3]:
import csv
import pandas as pd

In [4]:
import time

In [5]:
time.sleep(int(3600 *1.5))

In [6]:
fdeductiva = 'Fuentes_consolidadas(2).xlsx'
df_dentidad = pd.read_excel(fdeductiva)
df_dentidad.Entidad = df_dentidad.Entidad.fillna('')
df_dentidad.Entidad = df_dentidad.Entidad.str.replace(r" $","")
df_dentidad['Tipo de Fuentes MacroCat'] = df_dentidad['Tipo de Fuentes MacroCat'].fillna('')
df_dentidad.head()

,Entidad,Cargo Específico,Tipo de Fuentes MacroCat,Tipo de Fuente MicroCat,País,Lengua,Deductivo o Inductivo
0,ACHET,Asociación de empresarios,Business Source,Asociación de empresarios,Chile,Spanish,Deductivo
1,AEF,Asociación de empresarios,Business Source,Asociación de empresarios,Chile,Spanish,Deductivo
2,Apyme Chile,Asociación de empresarios,Business Source,Asociación de empresarios,Chile,Spanish,Deductivo
3,Asociación de emprendedores,Asociación de empresarios,Business Source,Asociación de empresarios,Chile,Spanish,Deductivo
4,Asech,Asociación de empresarios,Business Source,Asociación de empresarios,Chile,Spanish,Deductivo


In [7]:
df_dentidad.País = df_dentidad.País.str.replace("Korea", "South Korea").str.replace("South Korean", "South Korea")

In [8]:
df_dentidad.País.value_counts()

Chile             929
United Kingdom    528
Mexico            468
Spain             393
United States     359
Brazil            346
South Korea       311
Japan             250
Germany           236
TODOS             141
Name: País, dtype: int64

In [9]:
country_dfs = {}

In [10]:

for el in df_dentidad.País.unique():
    print(str(el), str(el) == 'nan')
    if str(el) in 'nan':
        continue
    country_dfs[el] = df_dentidad[df_dentidad['País'] == el]
    

Chile False
Mexico False
Spain False
United States False
Brazil False
United Kingdom False
Germany False
South Korea False
Japan False
TODOS False


In [11]:
country_dfs.keys()

dict_keys(['Chile', 'Mexico', 'Spain', 'United States', 'Brazil', 'United Kingdom', 'Germany', 'South Korea', 'Japan', 'TODOS'])

In [12]:
'dem' in ['epidemic']

False

In [13]:
'dem' in 'this is an epidemic'.split()

False

In [14]:
df_dentidad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3961 entries, 0 to 3960
Data columns (total 7 columns):
Entidad                     3961 non-null object
Cargo Específico            2222 non-null object
Tipo de Fuentes MacroCat    3961 non-null object
Tipo de Fuente MicroCat     2390 non-null object
País                        3961 non-null object
Lengua                      3823 non-null object
Deductivo o Inductivo       3800 non-null object
dtypes: object(7)
memory usage: 216.7+ KB


In [15]:
import re

In [16]:
def clean(arr):
    newWord = ''
    for i in range(len(arr)):
        w = ',' if i > 0 else ''
        newWord+=w+arr[i]
    arr=newWord
    return arr

def strip_accents(text):
    aux_text = str(text).replace("á","a")
    aux_text = aux_text.replace("é","e")
    aux_text = aux_text.replace("í","i")
    aux_text = aux_text.replace("ó","o")
    aux_text = aux_text.replace("ú","u")
    aux_text = aux_text.replace("ñ","n")
    
    return aux_text

def isOnIt(word, sentence, flag_inductive = False, dec_or_ind = ''):
    word = str(word)
    
    sentence = hashtag_rex.sub("",str(sentence))
    sentence = spaces_rex.sub(" ",sentence)
    if not(word.isupper()):
        temp_words = word.upper()
    else:
        temp_words = word
    f_words = temp_words
    if dec_or_ind == 'Inductivo':
        f_words = f_words.replace(" "," [\w+ ]*")
    temp_words = temp_words.split(" ")
    if (len(temp_words ) == 2) and not(flag_inductive):
        regex_str = "\\b({}|{})\\b".format(f_words," ".join(temp_words[1:]))
    else:
        regex_str = "\\b({})\\b".format(f_words)
    if not(word.isupper()):
        x = re.findall(regex_str,sentence.upper())
    else:
        x = re.findall(regex_str,sentence)
        
    if x:
        return True
    else:
        return False
    

In [17]:
test = "Terremoto sacude las costas de Jamaica y Cuba. El sismo tuvo una magnitud de 7.7. Además, Minsal descartó presencia de Coronavirus en Chile tras el estudio a pacientes con síntomas. Y el Presidente Piñera llegó hasta las zonas más afectadas por las lluvias y aluviones en el norte de nuestro país."
test

'Terremoto sacude las costas de Jamaica y Cuba. El sismo tuvo una magnitud de 7.7. Además, Minsal descartó presencia de Coronavirus en Chile tras el estudio a pacientes con síntomas. Y el Presidente Piñera llegó hasta las zonas más afectadas por las lluvias y aluviones en el norte de nuestro país.'

In [18]:
'WHO'.isupper()

True

In [19]:
with dview.sync_imports():
    import re

importing re on engine(s)


In [20]:
hashtag_rex = re.compile("(#[A-Za-z0-9]+)")
spaces_rex = re.compile("\s\s+")

In [21]:
strip_accents(test)

'Terremoto sacude las costas de Jamaica y Cuba. El sismo tuvo una magnitud de 7.7. Ademas, Minsal descarto presencia de Coronavirus en Chile tras el estudio a pacientes con sintomas. Y el Presidente Pinera llego hasta las zonas mas afectadas por las lluvias y aluviones en el norte de nuestro pais.'

In [22]:
isOnIt("Sebastian Piñera",test)

True

In [23]:
ar = dview.push({'isOnIt':isOnIt,'clean':clean,'country_dfs':country_dfs,'strip_accents':strip_accents,'hashtag_rex':hashtag_rex,'spaces_rex':spaces_rex},block=False)
ar.wait_interactive()

  95/95 tasks finished after    0 s
done


In [24]:
test = dview.pull('country_dfs',block=True,targets=0)
test.keys()

dict_keys(['Brazil', 'Chile', 'Germany', 'Japan', 'Mexico', 'South Korea', 'Spain', 'TODOS', 'United Kingdom', 'United States'])

In [25]:
cFile01 = 'COVID-post_enero-julio_FBK-DEF_EM_NV.csv'
total_actores = []
total_tipos = []
df_test = pd.read_csv(cFile01, sep=';', low_memory=False)
print(df_test.info())

df_test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304002 entries, 0 to 304001
Data columns (total 9 columns):
Date                304002 non-null object
Country             304002 non-null object
MediaOutlet         304002 non-null object
MediaOrigin         304002 non-null object
Message             296906 non-null object
LinkText            293995 non-null object
Verbs R2            90521 non-null object
Actors R1           40416 non-null object
Actors R1 - Type    40416 non-null object
dtypes: object(9)
memory usage: 20.9+ MB
None


,Date,Country,MediaOutlet,MediaOrigin,Message,LinkText,Verbs R2,Actors R1,Actors R1 - Type
0,"Jan 2, 2020 @ 19:19:38.000",United States,Los Angeles Times,Print,The Trump administration has announced a tempo...,"Trump administration imposed ban on some, but ...",announced,NaN,NaN
1,"Jan 2, 2020 @ 22:39:39.000",United States,CBS US,Radio,"The U.S. faces an epidemic of cyberattacks, wi...",Ransomware attack shuts down some Michigan sch...,NaN,NaN,NaN
2,"Jan 3, 2020 @ 01:40:00.000",United States,Los Angeles Times,Print,California has expanded a quarantine zone in S...,California expands local quarantine to halt ci...,state,NaN,NaN
3,"Jan 3, 2020 @ 08:16:13.000",Spain,24hTVE,TV,"La decisión, que entrará en vigor en 30 días, ...",Prohíben en Estados Unidos los cartuchos para ...,NaN,NaN,NaN
4,"Jan 3, 2020 @ 14:40:00.000",Chile,Elmostrador.cl,Online,CULTURA| La escasez hídrica producto del cambi...,Académica chilena en Nature: Más inversión en ...,NaN,NaN,NaN


In [26]:
i = 1067
df_test.iloc[3189,:]
country = df_test.iloc[i]['Country']
country_df = country_dfs[country]
country_df = country_df.loc[:,['Entidad', 'Tipo de Fuentes MacroCat','Deductivo o Inductivo']].values
print(df_test.iloc[i]['Message'])
print(len(country_df))
print()
actores = []
tipos = []

for j in range(len(country_df)):
    if not str(df_test.iloc[i]['Message']) in ['nan','--','/','-','?',',']:
        
        if isOnIt(country_df[j][0],df_test.iloc[i]['Message'],True,country_df[j][2]):
            print(country_df[j][0],country_df[j][2])
            print(df_test.iloc[i]['MediaOutlet'])
            print(country_df[j][0] in df_test.iloc[i]['MediaOutlet'])
            if country_df[j][0] not in actores and not(country_df[j][0] in df_test.iloc[i]['MediaOutlet']):
                actores.append(country_df[j][0])
                tipos.append(country_df[j][1])

In The New York Times Opinion Section "This virus is anything but patriotic. It is exploding in China and spreading to other countries, much as SARS did in 2003," writes Yanzhong Huang.
359

New York Times Deductivo
New York Times
True


In [27]:
actores

[]

In [28]:
for i in (range(3185,3195)):
    actores = []
    tipos = []
    country = df_test.iloc[i]['Country']
    country_df = country_dfs[country]
    country_df = country_df.loc[:,['Entidad','Tipo de Fuentes MacroCat','Deductivo o Inductivo','Cargo Específico']].values
    print(df_test.iloc[i]['Message'])
    for j in range(len(country_df)):
        if not str(df_test.iloc[i]['Message']) in ['nan','--','/','-','?',',']:
            if isOnIt(country_df[j][0],df_test.iloc[i]['Message'],True,country_df[j][2]):
                if country_df[j][0] not in actores and not(country_df[j][0].lower() in df_test.iloc[i]['MediaOutlet'].lower()):
                    actores.append(country_df[j][0])
                    tipos.append(country_df[j][1])
            if isOnIt(country_df[j][3],df_test.iloc[i]['Message'],True,country_df[j][2]):
                if country_df[j][3] not in actores and not(country_df[j][3].lower() in df_test.iloc[i]['MediaOutlet'].lower()):
                    actores.append(country_df[j][3])
                    tipos.append(country_df[j][1])
                    
    todos_df = country_dfs["TODOS"]
    todos_df = todos_df.loc[:,['Entidad', 'Tipo de Fuentes MacroCat','Deductivo o Inductivo','Cargo Específico']].values
    for j in range(len(todos_df)):
        if not str(df_test.iloc[i]['Message']) in ['nan','--','/','-','?',',']:
            if isOnIt(todos_df[j][0],df_test.iloc[i]['Message'],True,todos_df[j][2]):
                if todos_df[j][0] not in actores and not(todos_df[j][0].lower() in df_test.iloc[i]['MediaOutlet'].lower()):
                    actores.append(todos_df[j][0])
                    tipos.append(todos_df[j][1])
            if isOnIt(todos_df[j][3],df_test.iloc[i]['Message'],True,todos_df[j][2]):
                if todos_df[j][3] not in actores and not(todos_df[j][3].lower() in df_test.iloc[i]['MediaOutlet'].lower()):
                    actores.append(todos_df[j][3])
                    tipos.append(todos_df[j][1])


    actores = clean(actores)
    tipos = clean(tipos)
    total_actores.append(actores)
    total_tipos.append(tipos)
    print(i)

The Coronavirus is coming from Wuhan to Ontario airport thanks to the federal government https://omny.fm/shows/the-drive-home-with-jillian-barberie-and-john-ph-1/the-john-phillips-show-01-28-20-12pm
3185
El sujeto se encuentra realizando los trámites para volver a su país desde la ciudad donde se originó el coronavirus. A pesar de esta cifra oficial, el gobernador de Chihuahua, Javier Corral, mencionó la existencia de otros compatriotas que quieren salir de capital china.
3186
🔴 ÚLTIMA HORA. Los cuatro afectados trabajan en la misma empresa donde había acudido hace unos días a impartir un curso de capacitación una mujer procedente de china.
3187
Coronavirus in the US: A map of where the few cases have been confirmed across the country.
3188
⭕️ Terremoto sacude las costas de Jamaica y Cuba. El sismo tuvo una magnitud de 7.7. Además, Minsal descartó presencia de Coronavirus en Chile tras el estudio a pacientes con síntomas. Y el Presidente Piñera llegó hasta las zonas más afectadas por l

In [29]:
total_actores

['Government',
 '',
 '',
 '',
 'Minsal,Presidente Piñera,presidente,piñera',
 '',
 '',
 'Secretario de Salud',
 '',
 '']

In [30]:
total_tipos[:5]

['Political and Partisian Source',
 '',
 '',
 '',
 'Political and Partisian Source,Political and Partisian Source,Political and Partisian Source,Political and Partisian Source']

In [31]:
isOnIt(df_dentidad.iloc[189]['Entidad'],df_test.iloc[i]['Message'])

False

In [32]:
ar = dview.push({'df_dentidad':df_dentidad,'df_test':df_test},block=False)
ar.wait_interactive()

  95/95 tasks finished after   69 s
done


In [33]:
from tqdm.notebook import tqdm

In [34]:
@bview.parallel()
def get_actors(idx):
    actores = []
    tipos = []
    country = df_test.iloc[idx]['Country']
    if 'Korea' in country:
        country = "South Korea"
    country_df = country_dfs[country]
    country_df = country_df.loc[:,['Entidad', 'Tipo de Fuentes MacroCat','Deductivo o Inductivo','Cargo Específico']].values
    #print(df_test.iloc[idx]['Message'])
    for j in range(len(country_df)):
        if not str(df_test.iloc[idx]['Message']) in ['nan','--','/','-','?',',']:
            if isOnIt(country_df[j][0],df_test.iloc[idx]['Message'],True,country_df[j][2]):
                if str(country_df[j][0]) not in actores and not(str(country_df[j][0]).lower() in df_test.iloc[idx]['MediaOutlet'].lower()):
                    actores.append(str(country_df[j][0]))
                    tipos.append(country_df[j][1])
            if isOnIt(country_df[j][3],df_test.iloc[idx]['Message'],True,country_df[j][2]):
                if str(country_df[j][3]) not in actores and not(str(country_df[j][3]).lower() in df_test.iloc[idx]['MediaOutlet'].lower()):
                    actores.append(str(country_df[j][3]))
                    tipos.append(country_df[j][1])
                    
    todos_df = country_dfs["TODOS"]
    todos_df = todos_df.loc[:,['Entidad', 'Tipo de Fuentes MacroCat','Deductivo o Inductivo','Cargo Específico']].values
    for j in range(len(todos_df)):
        if not str(df_test.iloc[idx]['Message']) in ['nan','--','/','-','?',',']:
            if isOnIt(todos_df[j][0],df_test.iloc[idx]['Message'],True,todos_df[j][2]):
                if todos_df[j][0] not in actores and not(str(todos_df[j][0]).lower() in df_test.iloc[idx]['MediaOutlet'].lower()):
                    actores.append(todos_df[j][0])
                    tipos.append(todos_df[j][1])
            if isOnIt(todos_df[j][3],df_test.iloc[idx]['Message'],True,todos_df[j][2]):
                if str(todos_df[j][3]) not in actores and not(str(todos_df[j][3]).lower() in df_test.iloc[idx]['MediaOutlet'].lower()):
                    actores.append(str(todos_df[j][3]))
                    tipos.append(todos_df[j][1])



    actores = clean(actores)
    tipos = clean(tipos)
    
    return actores, tipos

In [35]:
import numpy as np


In [36]:
country = df_test.iloc[5]['Country']
country

'Spain'

In [37]:
total_actores = []
total_tipos = []

res = get_actors.map(np.arange(0,len(df_test)))
res.wait_interactive()
ans = res.get()

for el in ans:
    total_actores.append(el[0])
    total_tipos.append(el[1])

304002/304002 tasks finished after 2008 s
done


In [38]:
total_actores[3185:3192]

['Government',
 '',
 '',
 '',
 'Minsal,Presidente Piñera,presidente,piñera',
 '',
 '']

In [39]:
country_dfs['South Korea'].head()

,Entidad,Cargo Específico,Tipo de Fuentes MacroCat,Tipo de Fuente MicroCat,País,Lengua,Deductivo o Inductivo
44,대한상공회의소,사업가,Business Source,Asociación de empresarios,South Korea,Korean,Deductivo
45,소상공인연합회,사업가,Business Source,Asociación de empresarios,South Korea,Korean,Deductivo
46,중견기업연합회,사업가,Business Source,Asociación de empresarios,South Korea,Korean,Deductivo
47,중소기업중앙회,사업가,Business Source,Asociación de empresarios,South Korea,Korean,Deductivo
48,한국경영자총협회,사업가,Business Source,Asociación de empresarios,South Korea,Korean,Deductivo


In [40]:
df_test['Actors R1'] = total_actores
df_test['Actors R1 - Type'] = total_tipos
df_test.to_csv(cFile01.replace("EM_NV","final"),encoding='utf-8-sig',sep=";",index=False)
df_test.to_excel(cFile01.replace("EM_NV","final").replace("csv","xlsx"),index=False, engine='xlsxwriter')

In [41]:
import gc

In [42]:
"""del res
del ans
del total_actores
del total_tipos
del df_test

for i in range(10): gc.collect()"""

'del res\ndel ans\ndel total_actores\ndel total_tipos\ndel df_test\n\nfor i in range(10): gc.collect()'

In [43]:
cFile01 = 'COVID-post_enero-julio_IGRAM-DEF_EM_NV.csv'
total_actores = []
total_tipos = []
df_test = pd.read_csv(cFile01, sep=';', low_memory=False)

ar = dview.push({'df_test':df_test},block=False)
ar.wait_interactive()

  95/95 tasks finished after   17 s
done


In [44]:
res = get_actors.map(range(len(df_test)))
res.wait_interactive()
ans = res.get()

for el in ans:
    total_actores.append(el[0])
    total_tipos.append(el[1])

33970/33970 tasks finished after  236 s
done


In [45]:
df_test['Actors R1'] = total_actores
df_test['Actors R1 - Type'] = total_tipos
df_test.to_csv(cFile01.replace("EM_NV","final"),encoding='utf-8-sig',sep=";",index=False)
df_test.to_excel(cFile01.replace("EM_NV","final").replace("csv","xlsx"),index=False, engine='xlsxwriter')

In [46]:
"""del res
del ans
del total_actores
del total_tipos
del df_test

for i in range(10): gc.collect()"""

'del res\ndel ans\ndel total_actores\ndel total_tipos\ndel df_test\n\nfor i in range(10): gc.collect()'

In [47]:
cFile01 = 'COVID-post_enero-julio_TWR-DEF_EM_NV.csv'
total_actores = []
total_tipos = []
df_test = pd.read_csv(cFile01, sep=';', low_memory=False)

ar = dview.push({'df_test':df_test},block=False)
ar.wait_interactive()

  95/95 tasks finished after   53 s
done


In [48]:
res = get_actors.map(range(len(df_test)))
res.wait_interactive()
ans = res.get()

for el in ans:
    total_actores.append(el[0])
    total_tipos.append(el[1])

314768/314768 tasks finished after 2020 s
done


In [49]:
df_test['Actors R1'] = total_actores
df_test['Actors R1 - Type'] = total_tipos
df_test.to_csv(cFile01.replace("EM_NV","final"),encoding='utf-8-sig',sep=";",index=False)
df_test.to_excel(cFile01.replace("EM_NV","final").replace("csv","xlsx"),index=False, engine='xlsxwriter')

In [50]:
del res
del ans
del total_actores
del total_tipos
del df_test
for i in range(30): gc.collect()

In [51]:
0/0

ZeroDivisionError: division by zero

In [ ]:
ans[11]

In [ ]:
df_test[df_test.Message == '--']

In [ ]:
for i in tqdm(range(len(df_test))):
    actores = []
    tipos = []
    for j in range(len(df_dentidad)):
        if not str(df_test.iloc[i]['Message']) in ['nan','--','/','-','?',',']:
            if isOnIt(df_dentidad.iloc[j]['Entidad'],df_test.iloc[i]['Message']):
                if df_dentidad.iloc[j]['Entidad'] not in actores:
                    actores.append(df_dentidad.iloc[j]['Entidad'])
                    tipos.append(df_dentidad.iloc[j]['Tipo de Fuentes MacroCat'])


    actores = clean(actores)
    tipos = clean(tipos)
    total_actores.append(actores)
    total_tipos.append(tipos)
    print(i)

df_test['Actors R1'] = total_actores
df_test['Actors R1 - Type'] = total_tipos
df_test.to_csv(ftest,sep=";",index=False)

In [ ]:
not str(df_test.iloc[i]['Message']) in ['nan','--','/','-','?',',']

In [ ]:
%%time
if not str(df_test.iloc[i]['Message']) in ['nan','--','/','-','?',',']:
    if isOnIt(df_dentidad.iloc[j]['Entidad'],df_test.iloc[i]['Message']):
        if df_dentidad.iloc[j]['Entidad'] not in actores:
            actores.append(df_dentidad.iloc[j]['Entidad'])
            tipos.append(df_dentidad.iloc[j]['Tipo de Fuentes MacroCat'])

In [ ]:
actores

In [ ]:
df_dentidad.iloc[j]['Entidad']

In [ ]:
a = isOnIt(df_dentidad.iloc[j]['Entidad'],df_test.iloc[i]['Message'])
print(a)